In [1]:
%pip install langchain langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
# loading the text file conversation
from langchain.document_loaders import TextLoader

# use os module to go through the directory and load the text files
import os 
# Store all loaded documents
documents = []

data_dir = "data"  # replace with your directory
# Loop through all files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_dir, filename)
        loader = TextLoader(file_path)
        documents.extend(loader.load())  # append loaded documents


In [3]:
# split the big text into smaller chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [4]:
# embedding the text chunks

from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_24230/2721590312.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embedding=embedding_model)
retriever = vectorstore.as_retriever()


In [6]:
from langchain.llms import Ollama

# update this to load the different model

# Initialize Ollama with the gemma3 model
llm = Ollama(model="gemma3")
  

/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_24230/1557086837.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3")


In [7]:
from langchain.chains import RetrievalQA
# RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Ask a question
query = "What is the summary of the text?"
response = rag_chain(query)

print("Answer:", response['result'])

/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_24230/2052742362.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain(query)


Answer: The text discusses several key concepts in the field of Artificial Intelligence (AI). It defines AI as the broader concept of machines performing intelligent tasks, and breaks it down into subsets like Machine Learning (ML) and Deep Learning (DL). Natural Language Processing (NLP) is a field within AI focused on enabling machines to understand and generate human language. The text also highlights ethical concerns surrounding AI, such as bias, job displacement, and privacy. Finally, it introduces Explainable AI (XAI) as a method for increasing trust and accountability in AI systems by providing understandable justifications for decisions.


# Extend the pipeline to maintain chat history and provide additional context

In [10]:
# history aware chat
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder


retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)


contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [13]:
from langchain_core.prompts import PromptTemplate

# Create a prompt that includes the required 'context' variable
stuff_prompt = PromptTemplate(
	input_variables=["context", "input"],
	template=(
		"Use the following pieces of context to answer the question at the end.\n"
		"If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n"
		"{context}\n\nQuestion: {input}\nHelpful Answer:"
	)
)

question_answer_chain = create_stuff_documents_chain(llm, stuff_prompt)

In [14]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [16]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []

question1 = "What is the summary of the text?"
message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})

In [17]:
message1["answer"]

'The text discusses several key concepts in the field of Artificial Intelligence (AI). It defines AI as the broader concept of machines performing intelligent tasks, and breaks it down into subsets like Machine Learning (ML) and Deep Learning (DL). It also details Natural Language Processing (NLP), which focuses on enabling machines to understand and generate human language. The Turing Test is presented as a benchmark for machine intelligence, and highlights ethical concerns surrounding AI, such as bias and lack of transparency. Finally, it introduces Explainable AI (XAI) as a method to improve trust and accountability in AI systems.'

In [18]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)

In [19]:
chat_history


[HumanMessage(content='What is the summary of the text?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The text discusses several key concepts in the field of Artificial Intelligence (AI). It defines AI as the broader concept of machines performing intelligent tasks, and breaks it down into subsets like Machine Learning (ML) and Deep Learning (DL). It also details Natural Language Processing (NLP), which focuses on enabling machines to understand and generate human language. The Turing Test is presented as a benchmark for machine intelligence, and highlights ethical concerns surrounding AI, such as bias and lack of transparency. Finally, it introduces Explainable AI (XAI) as a method to improve trust and accountability in AI systems.', additional_kwargs={}, response_metadata={})]

In [21]:

second_question = "provide the previous answer within 100 characters"
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])

Bias, job loss, privacy, misuse, and lack of transparency are ethical AI concerns.


In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [23]:
store = {}

In [24]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [25]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [26]:

conversational_rag_chain.invoke(
    {"input": "Summarize the text"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'The text describes several key areas within Artificial Intelligence (AI). AI is the broader concept of machines performing intelligent tasks. It encompasses various types, including Narrow AI (or Weak AI), which is specialized like recommendation engines and chatbots, and General AI, which would possess human-level intelligence. The text also discusses related fields like Machine Learning (ML) and Deep Learning (DL), which utilize neural networks. Furthermore, it highlights specific applications of AI such as virtual assistants, self-driving cars, fraud detection, medical diagnosis, and language translation, alongside the historical Turing Test, which assesses a machine’s ability to mimic human conversation.'

In [27]:

store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='Summarize the text', additional_kwargs={}, response_metadata={}), AIMessage(content='The text describes several key areas within Artificial Intelligence (AI). AI is the broader concept of machines performing intelligent tasks. It encompasses various types, including Narrow AI (or Weak AI), which is specialized like recommendation engines and chatbots, and General AI, which would possess human-level intelligence. The text also discusses related fields like Machine Learning (ML) and Deep Learning (DL), which utilize neural networks. Furthermore, it highlights specific applications of AI such as virtual assistants, self-driving cars, fraud detection, medical diagnosis, and language translation, alongside the historical Turing Test, which assesses a machine’s ability to mimic human conversation.', additional_kwargs={}, response_metadata={})])}

In [28]:
conversational_rag_chain.invoke(
    {"input": "provide the previous answer within 100 characters"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'AI encompasses narrow, general, and super AI types, with ethical concerns like bias and job displacement.'

In [29]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: Summarize the text

AI: The text describes several key areas within Artificial Intelligence (AI). AI is the broader concept of machines performing intelligent tasks. It encompasses various types, including Narrow AI (or Weak AI), which is specialized like recommendation engines and chatbots, and General AI, which would possess human-level intelligence. The text also discusses related fields like Machine Learning (ML) and Deep Learning (DL), which utilize neural networks. Furthermore, it highlights specific applications of AI such as virtual assistants, self-driving cars, fraud detection, medical diagnosis, and language translation, alongside the historical Turing Test, which assesses a machine’s ability to mimic human conversation.

User: provide the previous answer within 100 characters

AI: AI encompasses narrow, general, and super AI types, with ethical concerns like bias and job displacement.

